# Peripherals and the Pmod I/O overlay

The Zybo has 6 *Peripheral Module* (Pmod) interfaces available on the board. A Pmod interface is a 12-pin connector that can be used to connect peripherals and other accessories to the board. 

Pmod peripherals are available from [Digilent Inc](http://www.digilentinc.com) and other suppliers. 

## Pmod peripherals

![](./images/Pmods_opt.png)

### Supported peripherals
Pynq current supports a number of Pmods and Grove Peripherals. See the *pynq.pmods package* section of the documentation. 

Pmods can be plugged directly into the Pmod port. Grove Peripherals can be connected to the Pmod Port through a [*StickIt*](http://www.xess.com/shop/product/stickit-grove/) adapter board.

As the PMOD interface connects to FPGA pins, other peripherals can be wired to a Pmod port.

### Pmod Connector
Each Pmod connector has 12 pins (2 rows of 6 pins, where each row has 3.3V (VCC), ground (GND) and 4 data pins giving 8 data pins in total). 

*Close-up of a 12-pin Pmod*

![](images/pmod_closeup_opt.jpeg)

### Pmod pin numbers and assignment

![](images/pmod_pins_opt.png)

Note the Pmod data pins are labelled 0-7. 

### Pmod Pin configuration

Pmods come in different configurations depending on the number of data pins required (single row, double row, e.g. 1x6 pins, 2x4 pins, 2x6 pins). Pmod peripherals with only a single row of pins can be plugged into the top row or the bottom row of the Pmod connector.

Pmods that use both rows (e.g. 2x4 pins, 2x6 pins), should in general be aligned to the upper-left of the connector (to align with VCC and GND).

![](images/tmp2_8pin_opt.jpeg)

### Pmod-Grove Adapter

Each Grove connector has 4 pins, one each for VCC and GND leaving 2 pins on each connector for signal wires. The Pmod-to-Grove StickIt adapter has 4 connectors for Grove devices, GR1, GR2, GR3, GR4.

![](./images/Pmod_Grove_Adapter_opt.jpeg)

All pins operate at 3.3V. Due to different pull-up/pull-down I/O requirements for different peripherals (e.g. IIC requires pull-up, and SPI requires pull-down), Grove peripherals must be plugged into the appropriate connector.

GR 1 and 2 pins are connected to pins with pull-down resistors, and GR3 and GR4 are connected to pins with pull-up resistors, as indicated in the image. This doesn't affect Pmods.

![](./images/stickit_mapping_opt.jpg)

## IO Processors
For overlays to be useful, they must provide sufficient functionality, while also providing flexibility to suit a wide range of applications. Flexibility in the overlay is provided through IO Processors (IOPs). An IO Processor is implemented in the Programmable Logic (PL or FPGA fabric) and consists of a MicroBlaze processor, Memory, Timer, IIC, SPI, GPIO, Debug logic and a configurable switch.

![](images/iop.jpg)

The IOP gives flexibility to connect to and control a range of different external accessories without requiring a re-design of the FPGA hardware. The IOP can be used to control a Pmod port, or other interface, or to off-load applications from the main ARM processors in the Zynq PS.

Each IOP has a dedicated memory block for the MicroBlaze instruction and data memory. This memory block is dual-ported, with one port connected to the MicroBlaze, and the other connected to the Cortex-A9 processor. This allows the ARM processor to access the MicroBlaze memory and dynamically write a new program to the MicroBlaze instruction area. The data area can be used for communication and data transfer between the ARM processor and the IOP(s).

Currently, one IOP is dedicated to each Pmod port. Inside the IOP are dedicated interface blocks; Timer, IIC, SPI and GPIO (General Purpose Input/Output). IIC and SPI are specialized interfaces used by many of the available Pmod, Grove and other peripherals, and GPIO can be used to connect to custom interfaces or used as simple inputs and outputs. When a Pmod or other peripheral is plugged in to a port, the configurable switch allows the Pmod signals to be routed internally to the required interface block.

## Zybo Pmod I/O overlay

The Zybo has 6 Pmod connectors. Ports 1-4 as indicated below are connected the Zynq PL. XADC and JF are not included/connected in this overlay.


![](images/zybopmods_opt.jpeg)

The Pmod I/O overlay for the Zybo contains 4 IOPs each connected to one of the PL Pmod ports. This allows control of the Pmod from Python, via the IOP.


![](images/pmodio_overlay_1_opt.png)

## Using Pmods with an overlay
To use a peripheral two software components are required; a driver application written in C for the IOP, and a Python module. These components are provided as part of the Pynq pacakge for supported peripherals. See the *IO Processors: Writing your own software* section of the documentation for writing drivers for your own periphals. 

The Python module instantiates the peripheral, and loads the driver application to the appropriate IOP. The IOP will also be reset and start executing the new application. 

The Python module will send commands which the IOP, and read and write data from the shared memory area. 

## Example: Using the OLED and the Ambient Light Sensor (ALS)

This examples requires the PmodOLED (OLED), and PmodALS (Ambient Light Sensor). 
Plug the PmodOLED into Port 1, and PmodALS into the top row of Port 2. (Currently, the PmodALS can only be used in the top row of a Pmod port.)

OLED displaying light reading from ambient light sensor:

![](images/als_oled_Demo_opt.jpeg)

Execute the next cell to load the FPGA Overlay, and then import the OLED module and instantiate it on port 1/IOP 1:

In [1]:
from pynq import Overlay
from pynq.pmods import PMOD_OLED
ol = Overlay("pmod.bit")
ol.download()
oled = PMOD_OLED(1)

Try writing a message to the OLED.

In [2]:
oled.write("Hello World")

In [3]:
oled.clear()

Import the ALS library, create an instance of the ALS Pmod, and read the value from the sensor

In [4]:
from pynq.pmods import PMOD_ALS
als = PMOD_ALS(2)
als.read()

47

Write the value from the ALS to the OLED. The ALS sensor returns an 8-bit value. 0 = Darkest, 255 = Brightest

In [5]:
oled.write("Light value : " + str(als.read()))

In [6]:
from pynq.pmods import PMOD_ALS
import time
als = PMOD_ALS(2)
als.read()

als.set_log_interval_ms(100)

als.start_log() # start logging values at intervals of delay
time.sleep(1)
als.stop_log()
als.get_log()

[41, 33, 32, 32, 32, 16, 31, 31, 31, 31, 30]

## Example: Using the ADC, DAC and OLED
This example requires the PmodAD2 (Analog to Digital Converter), PmodDA4 (Digital to Analog Converter) and PmodOLED (OLED). You will connect the output of the DAC to the input of the ADC, write a value to the DAC which the ADC will measure, and finally, write the value to the OLED for display.

* Set the jumper (JP1) on the PmodAD2 to Ref

Insert the following Pmods:
* DAC to Port 1 (Top Row)
* ADC to Port 2 (Align to the left of the port)
* OLED to Port 4

* Connect a wire from PmodA4, pin: A to PmodAD2, pin: V1

![](./images/adc_dac_demo_opt.jpeg)

Execute the cell below. Note that if you executed the cells above, you already loaded the overlay. However, it is good practice to load your overlay at the start of your code each time. 

You can set a different DAC output voltae by modifying the value in the cell below (must be between 0 and 2.0) and executing the cell again. The return value from the ADC and displayed on the LED is the measured value. There may be some small error between this and the value written to the DAC. 

In [7]:
from pynq.pmods import PMOD_OLED, PMOD_ADC, PMOD_DAC
from pynq import Overlay
ol = Overlay("pmod.bit")
ol.download()

dac = PMOD_DAC(1)
adc = PMOD_ADC(2)
oled = PMOD_OLED(4)

dac.write(0.75) # Set the DAC output voltage
value = adc.read() # Measure output of DAC with ADC
oled.write(str(value)) # Display measured value on OLED

For information on other supported peripherals and their API, see the *Pmods pynq.pmods package* section of the documentation.

## Going further
Scripts can be created and executed from the notebook. 

In this example, the code below should be copied to a script and saved on the board. 

- In the Pynq Portal home area [pynq:9090](http://pynq:9090) go to the *Getting Started* folder and select New > Text File (which opens in a new window)
- Copy and paste the code below into the file
- Rename the file dac_loop.py (File > Rename), and save it (File Rename).
- In the Pynq portal Getting_Started folder verify the file exists

```python
from pynq import Overlay
import time
#### dac_loop.py
def dac_loop(DAC_Port = 1, ADC_Port = 2, OLED_Port = 4,Switch_nmbr = 3, delay_secs = 1):
    '''
    DAC -> ADC -> OLED -- LOOP OVER 10 Values
    '''
    ol = Overlay("pmod.bit")
    ol.download()
    # Import PMOD classes from library PythonXilinx (pynq.pmods)
    from pynq.pmods import PMOD_OLED, PMOD_ADC, PMOD_DAC

    from pynq.board import  LED, Switch

    # Match PMOD instances to PMOD Connectors

    # Instantiate OLED
    oled = PMOD_OLED(OLED_Port)

    # Instantiate Digital to Analog Converter
    dac = PMOD_DAC(DAC_Port)

    # Instantiate Analog to Digital Converter
    adc = PMOD_ADC(ADC_Port)

    LED(Switch_nmbr).write(0)
    
    if(Switch(Switch_nmbr).read() == 0):
        print("Switch 3 is off. To run the loop, set Switch 3 to on before running")
    else:
        print("Turn Switch 3 off to stop the program")
    
    while Switch(Switch_nmbr).read():
        LED(Switch_nmbr).write(1)
        for i in range (1, 11):
            dac.write(i/10)
            measured = adc.read()
            time.sleep(.1)
            print('Measurement {} equals: {}'.format(i, measured))
            oled.write(str(measured))
    LED(Switch_nmbr).write(0)
```

By default, the DAC is be connected to Port 1 (Top Row), the ADC is connected to Port 2, and OLED is connected to Port 4. The function reads  _Switch_ 3 on the board. If the switch is on, it will run a loop that writes to the DAC, reads the value on the ADC and writes to the OLED. This is the same functionality as the previous example, but this time executing from a script, and inside a loop controlled by a switch.

Set Switch 3 on the board to "on" (or up), and then execute the cell below. This will run the script which will load the dac_loop() function into the notebook, and then execute the dac_loop() function in the cell below. You can change the ports the Pmods are attached to by passing the port numbers as parameters to the function. 

In [4]:
%run dac_loop.py
dac_loop(1,2,4)

Turn Switch 3 off to stop the program
Measurement 1 equals: 0.1016
Measurement 2 equals: 0.1953
Measurement 3 equals: 0.3047
Measurement 4 equals: 0.3984
Measurement 5 equals: 0.4922
Measurement 6 equals: 0.5898
Measurement 7 equals: 0.6836
Measurement 8 equals: 0.7773
Measurement 9 equals: 0.8711
Measurement 10 equals: 1.0078
